<a href="https://colab.research.google.com/github/pradeepelavarasan/Image-Classification--Tiny-ImageNet/blob/master/Pradeep_Elavarasan_EIP3_Assignment4_Final_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing Required Packages**

In [0]:
from google.colab import drive

import cv2

#!pip uninstall keras-preprocessing
#!pip install git+https://github.com/keras-team/keras-preprocessing.git

import os
import matplotlib
import numpy as np
import pandas as pd
from scipy import ndimage
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import zipfile
import requests
from io import StringIO
from sklearn import preprocessing
import tensorflow
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.callbacks import *

import skimage

from tensorflow.python.keras.utils import np_utils

import pickle
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


from __future__ import print_function
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import *

from tensorflow.keras.models import load_model

# Setting up  TPU ------------------------------

import os

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  print(devices)
  
  tf.logging.set_verbosity(tf.logging.DEBUG)
  


TPU address is grpc://10.4.213.10:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4313406809111492259), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4506753524439952341), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17784538767339841784), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 390743458509973771), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 4248580409610210351), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 9465965433712178690), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 10770749135754516838), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 1462807666267413378), _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 9875596676998249099), _D

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)

# Loading data to Colab

In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!unzip '/content/gdrive/My Drive/Assignment_4/tiny-imagenet-200.zip'

from zipfile import ZipFile
with ZipFile('/content/gdrive/My Drive/Assignment_4/tiny-imagenet-200.zip') as zf:
  zf.extractall('/content')

#ZipFile.extract(member = '/content/gdrive/My Drive/Assignment_4/tiny-imagenet-200.zip')

# Functions required for building custom RESNET

In [0]:
from __future__ import division

import six
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    Activation,
    Dense,
    Flatten,
    Lambda
)
from tensorflow.python.keras.layers.convolutional import (
    Conv2D,
    MaxPooling2D,
    AveragePooling2D
)

from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, concatenate, SeparableConv2D
from tensorflow.python.keras.layers.merge import add
from tensorflow.python.keras.layers.normalization import BatchNormalization
from tensorflow.keras.regularizers import l2
from keras import backend as K


def _bn_relu(input):
    """Helper to build a BN -> relu block
    """
    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)
    activation = Activation("relu")(norm)
    return activation
    #return Dropout(0.2)(activation)


def _conv_bn_relu(filters=32):
    """Helper to build a conv -> BN -> relu block
    """
    def f(input):
        conv = SeparableConv2D(filters=filters, kernel_size=(3,3),
                      strides=(1,1), padding="same",
                      kernel_initializer="he_normal",
                      kernel_regularizer=l2(1.e-4))(input)
       
        
        return _bn_relu(conv)

    return f

"""
def _bn_relu_conv(filters=32): # not used
    #Helper to build a BN -> relu -> conv block.
    #This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf
    
    
    def f(input):
        activation = _bn_relu(input)
        return Conv2D(filters=filters, kernel_size=(3, 3),
                      strides=strides, padding="same",
                      kernel_initializer="he_normal",
                      kernel_regularizer=l2(1.e-4))(activation)

    return f
"""


def _shortcut(input, residual):
    #Adds a shortcut between input and residual block and merges them with "sum"
  
     #Expand channels of shortcut to match residual.
     #Stride appropriately to match residual (width, height)
     #Should be int if network architecture is correctly configured.
    input_shape = K.int_shape(input)
    residual_shape = K.int_shape(residual)
    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))
    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))
    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]

    shortcut = input
    # 1 X 1 conv if shape is different. Else identity.
    if stride_width > 1 or stride_height > 1 or not equal_channels:
        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],
                          kernel_size=(1, 1),
                          strides=(stride_width, stride_height),
                          padding="valid",
                          kernel_initializer="he_normal",
                          kernel_regularizer=l2(0.0001))(input)

    return add([shortcut, residual])


def basic_block(filters):
    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    """
    def f(input):

        conv1 = _conv_bn_relu(filters=filters)(input)
        residual = _conv_bn_relu(filters=filters)(conv1)
        #output = _shortcut(input, residual)
        output = concatenate([input, residual])
        return output


    return f
    


def bottleneck(filters):
    """Bottleneck architecture for > 34 layer resnet.
    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf
    Returns:
        A final conv layer of filters * 4
    """
    def f(input):

        bottleneck = Conv2D(filters=filters,
                           kernel_size=(1, 1),
                           strides=(1,1),
                           padding="same",
                           kernel_initializer="he_normal",
                           kernel_regularizer=l2(1e-4))(input)
        return _bn_relu(bottleneck)
      
    return f


def _handle_dim_ordering():
    global ROW_AXIS
    global COL_AXIS
    global CHANNEL_AXIS
    if K.image_dim_ordering() == 'tf':
        ROW_AXIS = 1
        COL_AXIS = 2
        CHANNEL_AXIS = 3
    else:
        CHANNEL_AXIS = 1
        ROW_AXIS = 2
        COL_AXIS = 3


def _get_block(identifier):
    if isinstance(identifier, six.string_types):
        res = globals().get(identifier)
        if not res:
            raise ValueError('Invalid {}'.format(identifier))
        return res
    return identifier

  
class ResnetBuilder(object):
    @staticmethod
    def build(input_shape, num_outputs, block_fn, repetitions):
        """Builds a custom ResNet like architecture.
        Args:
            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)
            num_outputs: The number of outputs at final softmax layer
            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.
                The original paper used basic_block for layers < 50
            repetitions: Number of repetitions of various block units.
                At each block unit, the number of filters are doubled and the input size is halved
        Returns:
            The keras `Model`.
        """
        _handle_dim_ordering()
        if len(input_shape) != 3:
            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")

        # Permute dimension order if necessary
        if K.image_dim_ordering() == 'tf':
            input_shape = (input_shape[1], input_shape[2], input_shape[0])

        # Load function from str if needed.
        block_fn = _get_block(block_fn)
        
        #--------Initialization Block 0 
        input = Input(shape=input_shape)
        block0 = _conv_bn_relu(filters=32)(input)
        block0 = basic_block(filters=64)(block0)
        block0 = basic_block(filters=128)(block0)
        block0 = basic_block(filters=256)(block0)
        
        # Intersection/Bottleneck Block 0
        
        bottleneck0 = bottleneck(filters=64)(block0)
        skip_connection0 = bottleneck0 
        
        #--------Resnet Block 1 
        
        block1 = basic_block(filters=64)(bottleneck0)
        block1 = basic_block(filters=128)(block1)
        block1 = basic_block(filters=256)(block1)
        block1 = basic_block(filters=512)(block1)

        # Intersection/Bottleneck Block 1
        
        bottleneck1 = bottleneck(filters=128)(block1)
        skip_connection1 = bottleneck1 
        
        #--------Resnet Block 2
        block2 = concatenate([skip_connection0, skip_connection1])
        block2 = basic_block(filters=128)(block2)
        block2 = basic_block(filters=256)(block2)
        block2 = basic_block(filters=512)(block2)
        block2 = basic_block(filters=1024)(block2)

        # Intersection/Bottleneck Block 2
        
        bottleneck2 = bottleneck(filters=256)(block2)
        skip_connection2 = bottleneck2 
        
        #--------Resnet Block 3 
        block3 = concatenate([skip_connection0, skip_connection1, skip_connection2])
        block3 = basic_block(filters=256)(block3)
        block3 = basic_block(filters=512)(block3)
        block3 = basic_block(filters=512)(block3)
        block3 = basic_block(filters=1024)(block3)

        # Intersection/Bottleneck Block 3
        
        bottleneck3 = bottleneck(filters=256)(block3)
        skip_connection3 = bottleneck3 
        
        #--------Resnet Block 4 
        
        block4 = concatenate([skip_connection0, skip_connection1, skip_connection2, skip_connection3])
        block4= MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="same", name="Maxpool")(block4)
        block4 = basic_block(filters=256)(block4)
        block4 = basic_block(filters=512)(block4)

        # Intersection/Bottleneck Block 4
        
        bottleneck4 = bottleneck(filters=200)(block4)
        
        # Global Average pooling
        gap = GlobalAveragePooling2D()(bottleneck4)

        #Softmax
        output = Activation('softmax')(gap)

        model = Model(inputs=input, outputs=output)
        return model

    @staticmethod
    def build_resnet_18(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])

    @staticmethod
    def build_resnet_34(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])

    @staticmethod
    def build_resnet_50(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 0, 0])
    @staticmethod
    def build_custom_resnet(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [4, 4, 5, 3])

    @staticmethod
    def build_resnet_101(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])

    @staticmethod
    def build_resnet_152(input_shape, num_outputs):
        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])

Using TensorFlow backend.


# Data Generator Setup

***Additional Data Augmentation using IMGAUG***

In [0]:
from imgaug import augmenters as iaa

#aug1 = iaa.GaussianBlur(sigma=(0, 2.0))
#aug2 = iaa.AdditiveGaussianNoise(scale=0.20 * 255)
aug3 = iaa.ChannelShuffle(p=1.0)
#aug4 = iaa.AddToHueAndSaturation((-50,50))
#aug5 = iaa.GammaContrast((0.5,1.75),per_channel=True)
#aug6 = PadToFixedSize(height'=height+32',width'=width+32', position='center')
aug7 = iaa.CoarseDropout(p=0.2, size_percent=0.02)




def additional_augmenation(image):
    #image = aug1.augment_image(image)
    #image = aug2.augment_image(image)
    image = aug3.augment_image(image)
    #image = aug4.augment_image(image)
    image = aug7.augment_image(image)
    #image = skimage.exposure.rescale_intensity(aug5.augment_image(image),in_range=(0, 255))
    return image

**Loading of training and validation data**

In [0]:
train_datagen = ImageDataGenerator(
       #featurewise_center=False,  # set input mean to 0 over the dataset
      # samplewise_center=False,  # set each sample mean to 0
      #  featurewise_std_normalization=False,  # divide inputs by std of the dataset
      #  samplewise_std_normalization=False,  # divide each input by its std
      #  zca_whitening=False,  # apply ZCA whitening
      #  rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
      #  width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
       # height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        zoom_range=[0.5, 1.5], #zoom
      #  vertical_flip=False, # randomly flip images
        rescale= 1./255,
        preprocessing_function=additional_augmenation)  


train_generator = train_datagen.flow_from_directory(
    directory="./tiny-imagenet-200/train/",
    target_size=(32, 32),
    color_mode="rgb",
    batch_size=250,
    class_mode="categorical",
    shuffle=True,
    seed=1
)

Found 100000 images belonging to 200 classes.


In [0]:
val_data = pd.read_csv("./tiny-imagenet-200/val/val_annotations.txt", sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

valid_datagen=ImageDataGenerator(rescale= 1./255)
valid_generator=valid_datagen.flow_from_dataframe(dataframe=val_data, directory="./tiny-imagenet-200/val/images/", x_col="File", y_col="Class", class_mode="categorical",validate_filenames=False, target_size=(32,32), batch_size=250,seed=1)

Found 10000 images belonging to 200 classes.


# **Few Training Initializations**

In [0]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping

#lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), cooldown=0,verbose=1, patience=10, min_lr=0.5e-6)
#early_stopper = EarlyStopping(min_delta=0.001, patience=10)

STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
STEP_SIZE_VALID = valid_generator.n//valid_generator.batch_size



**Setting up Model weights back mechanism**

In [0]:
from tensorflow.keras.callbacks import *
filepath="/content/gdrive/My Drive/Assignment_4/Training_Backup/V34-epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

# Model Training on 32*32 Images

**Model building**

In [0]:
# input image dimensions
img_rows, img_cols = 32, 32
# The imagenet images are RGB.
nb_classes = 200
img_channels = 3

model = ResnetBuilder.build_custom_resnet((img_channels, img_rows, img_cols), nb_classes)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 32, 32, 32)   155         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 32, 32, 32)   128         separable_conv2d[0][0]           
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 32)   0           batch_normalization_v1[0][0]     
_____________________________________

**Compile**

In [0]:
adam = tf.train.AdamOptimizer(0.001)

#adam = tf.keras.Adam(learning_rate=0.001)

#adam = tf.keras.optimizers.Adam(lr=0.001) 

model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=adam,
              metrics=['accuracy'])


In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)
model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.62.229.122:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 1409611887889514383)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 9204565535668976996)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 11178418307169434886)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:

**Training with 32*32 epoch 1-25**

In [0]:
#model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/V28_desnse-epochs:023-val_acc:0.476.hdf5")
#model = load_model("/content/gdrive/My Drive/Assignment_4/Training_Backup/V4-TPU-resnet50-epochs:002-val_acc:0.168.hdf5")
last_executed_epoch = 0
nb_epoch = 25
csv_logger = CSVLogger('Assignment4_V34_32x32_1_25.csv')


In [0]:
    print('Training on 32*32 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    # This will do preprocessing and realtime data augmentation:

    # Fit the model on the batches generated by datagen.flow().
    #model.fit_generator(generator=train_generator,
     #                   steps_per_epoch=STEP_SIZE_TRAIN,               
      #                  epochs=nb_epoch,
       #                 initial_epoch=last_executed_epoch,verbose=1)
    
    
    model.fit_generator(generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])
    
    model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_1_25.hdf5")
    !cp "Assignment4_V34_32x32_1_25.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_1_25.csv"
    print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')
    

Training on 32*32 images: Epochs 1 - 25 : 
Epoch 1/25
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(31,), dtype=tf.int32, name='core_id_20'), TensorSpec(shape=(31, 32, 32, 3), dtype=tf.float32, name='input_2_10'), TensorSpec(shape=(31, 200), dtype=tf.float32, name='activation_81_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_2
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 86.66216588020325 secs
INFO:tensorflow:Setting weights on TPU model.
399/400 [============================>.] - ETA: 1s - loss: 5.0722 - acc: 0.0369INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(31,), dtype=tf.int32, name='core_id_30'), TensorSpec(shape=(31, 32, 32, 3), dtype=tf.float32, name='input_2_10'), TensorSpec(shape=(31, 200), dtype=tf.float32, name='activation_81_target_30')]
INFO:tensorflow:Overriding default plac

**Training with 32*32 epoch 26-50**

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_1_25.hdf5")
last_executed_epoch = 25
nb_epoch = 50
csv_logger = CSVLogger('Assignment4_V34_32x32_26_50.csv')


In [0]:
    print('Training on 32*32 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    # This will do preprocessing and realtime data augmentation:

    # Fit the model on the batches generated by datagen.flow().
    #model.fit_generator(generator=train_generator,
     #                   steps_per_epoch=STEP_SIZE_TRAIN,               
      #                  epochs=nb_epoch,
       #                 initial_epoch=last_executed_epoch,verbose=1)
    
    
    model.fit_generator(generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])
    model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_26_50.hdf5")
    !cp "Assignment4_V34_32x32_26_50.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_26_50.csv"
    print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 32*32 images: Epochs 26 - 50 : 
Epoch 26/50
INFO:tensorflow:Setting weights on TPU model.
400/400 [==============================] - 352s 880ms/step - loss: 2.7419 - acc: 0.3884 - val_loss: 3.4582 - val_acc: 0.3086
Epoch 27/50
400/400 [==============================] - 351s 877ms/step - loss: 2.7085 - acc: 0.3936 - val_loss: 3.1029 - val_acc: 0.3649
Epoch 28/50
400/400 [==============================] - 352s 881ms/step - loss: 2.6837 - acc: 0.3987 - val_loss: 3.0603 - val_acc: 0.3620
Epoch 29/50
400/400 [==============================] - 350s 875ms/step - loss: 2.6516 - acc: 0.4054 - val_loss: 3.0632 - val_acc: 0.3641
Epoch 30/50
400/400 [==============================] - 349s 871ms/step - loss: 2.6240 - acc: 0.4119 - val_loss: 2.8576 - val_acc: 0.3908
Epoch 31/50
400/400 [==============================] - 348s 870ms/step - loss: 2.6028 - acc: 0.4152 - val_loss: 3.1826 - val_acc: 0.3495
Epoch 32/50
400/400 [==============================] - 349s 873ms/step - loss: 2.5802 - 

**Training with 32*32 epoch 51-70**

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_26_50.hdf5")
last_executed_epoch = 50
nb_epoch = 70
csv_logger = CSVLogger('Assignment4_V34_32x32_51_75.csv')

In [0]:
    print('Training on 32*32 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    # This will do preprocessing and realtime data augmentation:

    # Fit the model on the batches generated by datagen.flow().
    #model.fit_generator(generator=train_generator,
     #                   steps_per_epoch=STEP_SIZE_TRAIN,               
      #                  epochs=nb_epoch,
       #                 initial_epoch=last_executed_epoch,verbose=1)
    
    
    model.fit_generator(generator=train_generator,
                        steps_per_epoch=STEP_SIZE_TRAIN,
                        validation_data=valid_generator,
                        validation_steps=STEP_SIZE_VALID,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])
    model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_51_70.hdf5")
    !cp "Assignment4_V34_32x32_51_75.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_51_75.csv"
    print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 32*32 images: Epochs 51 - 70 : 
Epoch 51/70
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(31,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(31, 32, 32, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(31, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 60.4656617641449 secs
INFO:tensorflow:Setting weights on TPU model.
399/400 [============================>.] - ETA: 1s - loss: 2.2002 - acc: 0.4974INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(31,), dtype=tf.int32, name='core_id_10'), TensorSpec(shape=(31, 32, 32, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(31, 2

# Model Training on 64*64 images

**Generation of 64*64 images**

In [0]:
train_generator_64 = train_datagen.flow_from_directory(
    directory="./tiny-imagenet-200/train/",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=128,
    class_mode="categorical",
    shuffle=True,
    seed=1
)
valid_generator_64=valid_datagen.flow_from_dataframe(dataframe=val_data, directory="./tiny-imagenet-200/val/images/", x_col="File", y_col="Class", class_mode="categorical",validate_filenames=False, shuffle=True, target_size=(64,64), batch_size=128,seed=1)

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
STEP_SIZE_TRAIN_64 = train_generator_64.n//train_generator_64.batch_size
STEP_SIZE_VALID_64 = valid_generator_64.n//valid_generator_64.batch_size

**Rebuilding the model to accept 64*64 images**

In [0]:
# input image dimensions
img_rows, img_cols = 64, 64
# The imagenet images are RGB.
nb_classes = 200
img_channels = 3

model = ResnetBuilder.build_custom_resnet((img_channels, img_rows, img_cols), nb_classes)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
separable_conv2d (SeparableConv (None, 64, 64, 32)   155         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_v1 (BatchNo (None, 64, 64, 32)   128         separable_conv2d[0][0]           
__________________________________________________________________________________________________
activation (Activation)         (None, 64, 64, 32)   0           batch_normalization_v1[0][0]     
_____________________________________

In [0]:
adam = tf.train.AdamOptimizer(0.001)
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=adam,
              metrics=['accuracy'])

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)
model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.42.191.74:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 14709204905424248740)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 12068932762043815323)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 7941128398303201474)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:

**Training with 64*64 epoch 71-80** (batchsize-100)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_32x32_51_70.hdf5")
last_executed_epoch = 70
nb_epoch = 80
csv_logger = CSVLogger('Assignment4_V34_64x64_71_80.csv')


In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_71_80.hdf5")
!cp "Assignment4_V34_64x64_71_80.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_71_80.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 71 - 80 : 
Epoch 71/80
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(12,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(12, 64, 64, 3), dtype=tf.float32, name='input_2_10'), TensorSpec(shape=(12, 200), dtype=tf.float32, name='activation_81_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_2
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 67.49458765983582 secs
INFO:tensorflow:Setting weights on TPU model.
 999/1000 [============================>.] - ETA: 1s - loss: 2.6700 - acc: 0.4350INFO:tensorflow:New input shapes; (re-)compiling: mode=eval (# of cores 8), [TensorSpec(shape=(12,), dtype=tf.int32, name='core_id_10'), TensorSpec(shape=(12, 64, 64, 3), dtype=tf.float32, name='input_2_10'), TensorSpec(shape=(12

**Training with 64*64 epoch 81-90** (batchsize-128)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_71_80.hdf5")
last_executed_epoch = 80
nb_epoch = 90
csv_logger = CSVLogger('Assignment4_V34_64x64_81_90.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_81_90.hdf5")
!cp "Assignment4_V34_64x64_81_90.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_81_90.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 81 - 90 : 
Epoch 81/90
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 53.63751268386841 secs
INFO:tensorflow:Setting weights on TPU model.
154/782 [====>.........................] - ETA: 26:03 - loss: 2.0246 - acc: 0.5502INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default place

**Training with 64*64 epoch 91-100** (batchsize-128)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_81_90.hdf5")
last_executed_epoch = 90
nb_epoch = 100
csv_logger = CSVLogger('Assignment4_V34_64x64_91_100.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_91_100.hdf5")
!cp "Assignment4_V34_64x64_91_100.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_91_100.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 91 - 100 : 
Epoch 91/100
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 66.30171728134155 secs
INFO:tensorflow:Setting weights on TPU model.
764/782 [============================>.] - ETA: 33s - loss: 1.8136 - acc: 0.5925INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4, 2

# Additional Augmentations

In [0]:
from imgaug import augmenters as iaa

#aug1 = iaa.GaussianBlur(sigma=(0, 2.0))
#aug2 = iaa.AdditiveGaussianNoise(scale=0.20 * 255)

#aug3 = iaa.ChannelShuffle(p=1.0) # multiple if better for colour adjustments
#aug4 = iaa.AddToHueAndSaturation((-50,50))
#aug5 = iaa.GammaContrast((0.5,1.75),per_channel=True)
#aug6 = PadToFixedSize(height'=height+32',width'=width+32', position='center')
#aug7 = iaa.CoarseDropout(p=0.2, size_percent=0.02)
#aug = iaa.Fliplr(0.5)
#aug = iaa.Superpixels(p_replace=0.5, n_segments=64)
#aug = iaa.Grayscale(alpha=(0.0, 1.0)) # Multiple is better for colour
#aug = iaa.Add((-40, 40), per_channel=0.5) # Multiply is better than add for diff colours
#aug = iaa.Multiply((0.5, 1.5), per_channel=0.5) # multiple colours
#aug = iaa.Affine(scale=(0.5, 1.5)) # zoom
#aug = iaa.Affine(rotate=(-45, 45)) # rotate
#aug = iaa.Noop() # No operation

# Below are 5 augmentation techniques used
custom_aug = iaa.Sequential([
      iaa.Fliplr(0.5),
      iaa.Multiply((0.5, 1.5), per_channel=0.5),
      iaa.Affine(scale=(0.25, 2.0)),
      iaa.Sometimes(0.7,iaa.CoarseDropout(p=0.2, size_percent=0.02),iaa.Affine(rotate=(-45, 45)))
      ], random_order=False)


def additional_augmenation(image):
    #image = aug1.augment_image(image)
    #image = aug2.augment_image(image)
    image = custom_aug.augment_image(image)
    #image = aug4.augment_image(image)
    #image = aug7.augment_image(image)
    #image = skimage.exposure.rescale_intensity(aug5.augment_image(image),in_range=(0, 255))
    return image
  
train_datagen = ImageDataGenerator(
       #featurewise_center=False,  # set input mean to 0 over the dataset
      # samplewise_center=False,  # set each sample mean to 0
      #  featurewise_std_normalization=False,  # divide inputs by std of the dataset
      #  samplewise_std_normalization=False,  # divide each input by its std
      #  zca_whitening=False,  # apply ZCA whitening
      #  rotation_range=20,  # randomly rotate images in the range (degrees, 0 to 180)
      #  width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
       # height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
       # horizontal_flip=True,  # randomly flip images
       # zoom_range=[0.5, 1.5], #zoom
      #  vertical_flip=False, # randomly flip images
        rescale= 1./255,
        preprocessing_function=additional_augmenation)  



val_data = pd.read_csv("./tiny-imagenet-200/val/val_annotations.txt", sep='\t', header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])
val_data.drop(['X', 'Y', 'H', 'W'], axis=1, inplace=True)

valid_datagen=ImageDataGenerator(rescale= 1./255)



**Setup required after addition augmentation changes**

In [0]:
train_generator_64 = train_datagen.flow_from_directory(
    directory="./tiny-imagenet-200/train/",
    target_size=(64, 64),
    color_mode="rgb",
    batch_size=128,
    class_mode="categorical",
    shuffle=True,
   # save_to_dir="/content/gdrive/My Drive/Assignment_4/Augmented_Images/Additional",
    seed=1
)
valid_generator_64=valid_datagen.flow_from_dataframe(dataframe=val_data, directory="./tiny-imagenet-200/val/images/", x_col="File", y_col="Class", class_mode="categorical",validate_filenames=False, shuffle=True, target_size=(64,64), batch_size=128,seed=1)

STEP_SIZE_TRAIN_64 = train_generator_64.n//train_generator_64.batch_size
STEP_SIZE_VALID_64 = valid_generator_64.n//valid_generator_64.batch_size

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [0]:
# input image dimensions
img_rows, img_cols = 64, 64
# The imagenet images are RGB.
nb_classes = 200
img_channels = 3

model = ResnetBuilder.build_custom_resnet((img_channels, img_rows, img_cols), nb_classes)
#model.summary() #Model summary is same as before so not printing it here agian.


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
adam = tf.train.AdamOptimizer(0.001)
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=adam,
              metrics=['accuracy'])

In [0]:
TPU_WORKER = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tf.logging.set_verbosity(tf.logging.INFO)
model = tf.contrib.tpu.keras_to_tpu_model(
    model,
    strategy=tf.contrib.tpu.TPUDistributionStrategy(
        tf.contrib.cluster_resolver.TPUClusterResolver(TPU_WORKER)))


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.4.213.10:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 4313406809111492259)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4506753524439952341)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 17784538767339841784)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1,

# Model Training on 64*64 images - with Additional Augmentations

**Training with 64*64 epoch 100-110** (batchsize-128)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V34_64x64_91_100.hdf5")
last_executed_epoch = 100
nb_epoch = 110
csv_logger = CSVLogger('Assignment4_V36_64x64_101_110.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_101_110.hdf5")
!cp "Assignment4_V36_64x64_101_110.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_101_110.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 101 - 110 : 
Epoch 101/110
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 65.42139983177185 secs
INFO:tensorflow:Setting weights on TPU model.
454/782 [================>.............] - ETA: 10:57 - loss: 2.5996 - acc: 0.4344INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(

**Training with 64*64 epoch 111-120** (batchsize-128)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_101_110.hdf5")
last_executed_epoch = 110
nb_epoch = 120
csv_logger = CSVLogger('Assignment4_V36_64x64_111_120.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_111_120.hdf5")
!cp "Assignment4_V36_64x64_111_120.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_111_120.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 111 - 120 : 
Epoch 111/120
INFO:tensorflow:Setting weights on TPU model.
782/782 [==============================] - 1353s 2s/step - loss: 2.1091 - acc: 0.5288 - val_loss: 2.4985 - val_acc: 0.5245
Epoch 112/120
782/782 [==============================] - 1331s 2s/step - loss: 2.0838 - acc: 0.5335 - val_loss: 2.5750 - val_acc: 0.5169
Epoch 113/120
782/782 [==============================] - 1328s 2s/step - loss: 2.0672 - acc: 0.5392 - val_loss: 2.3251 - val_acc: 0.5608
Epoch 114/120
782/782 [==============================] - 1331s 2s/step - loss: 2.0612 - acc: 0.5393 - val_loss: 2.4893 - val_acc: 0.5344
Epoch 115/120
782/782 [==============================] - 1330s 2s/step - loss: 2.0432 - acc: 0.5419 - val_loss: 2.3674 - val_acc: 0.5456
Epoch 116/120
427/782 [===============>..............] - ETA: 9:54 - loss: 2.0135 - acc: 0.5495Training on 64*64 images: Epochs 111 - 120 : 
Epoch 111/120
INFO:tensorflow:Setting weights on TPU model.
782/782 [=============

**Training with 64*64 epoch 121-130** (batchsize-128)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_111_120.hdf5")
last_executed_epoch = 120
nb_epoch = 130
csv_logger = CSVLogger('Assignment4_V36_64x64_121_130.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_121_130.hdf5")
!cp "Assignment4_V36_64x64_121_130.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_121_130.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 121 - 130 : 
Epoch 121/130
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 65.5228705406189 secs
INFO:tensorflow:Setting weights on TPU model.
244/782 [========>.....................] - ETA: 20:41 - loss: 1.9468 - acc: 0.5657INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4

**Training with 64*64 epoch 131-140** (batchsize-128)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_121_130.hdf5")
last_executed_epoch = 130
nb_epoch = 140
csv_logger = CSVLogger('Assignment4_V36_64x64_131_140.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_131_140.hdf5")
!cp "Assignment4_V36_64x64_131_140.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_131_140.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 131 - 140 : 
Epoch 131/140
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 65.72786903381348 secs
INFO:tensorflow:Setting weights on TPU model.
389/782 [=============>................] - ETA: 13:26 - loss: 1.8487 - acc: 0.5843INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(

**Training with 64*64 epoch 141-150** (batchsize-128)

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_131_140.hdf5")
last_executed_epoch = 140
nb_epoch = 150
csv_logger = CSVLogger('Assignment4_V36_64x64_141_150.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_141_150.hdf5")
!cp "Assignment4_V36_64x64_141_150.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_141_150.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 141 - 150 : 
Epoch 141/150
INFO:tensorflow:Setting weights on TPU model.
782/782 [==============================] - 1348s 2s/step - loss: 1.7723 - acc: 0.6007 - val_loss: 2.4050 - val_acc: 0.5507
Epoch 142/150
782/782 [==============================] - 1324s 2s/step - loss: 1.7590 - acc: 0.6032 - val_loss: 2.2962 - val_acc: 0.5743
Epoch 143/150
782/782 [==============================] - 1323s 2s/step - loss: 1.7483 - acc: 0.6049 - val_loss: 2.4032 - val_acc: 0.5457
Epoch 144/150
782/782 [==============================] - 1321s 2s/step - loss: 1.7449 - acc: 0.6042 - val_loss: 2.2869 - val_acc: 0.5624
Epoch 145/150
782/782 [==============================] - 1320s 2s/step - loss: 1.7350 - acc: 0.6075 - val_loss: 2.3589 - val_acc: 0.5494
Epoch 146/150
782/782 [==============================] - 1322s 2s/step - loss: 1.7265 - acc: 0.6096 - val_loss: 2.4209 - val_acc: 0.5544
Epoch 147/150
782/782 [==============================] - 1326s 2s/step - loss: 1.7150 

**Training with 64*64 epoch 151-160** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_141_150.hdf5")
last_executed_epoch = 150
nb_epoch = 160
csv_logger = CSVLogger('Assignment4_V36_64x64_151_160.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_151_160.hdf5")
!cp "Assignment4_V36_64x64_151_160.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_151_160.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 151 - 160 : 
Epoch 151/160
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 64.70670461654663 secs
INFO:tensorflow:Setting weights on TPU model.
627/782 [=======================>......] - ETA: 4:56 - loss: 1.6882 - acc: 0.6191INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4

**Training with 64*64 epoch 161-170** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_151_160.hdf5")
last_executed_epoch = 160
nb_epoch = 170
csv_logger = CSVLogger('Assignment4_V36_64x64_161_170.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_161_170.hdf5")
!cp "Assignment4_V36_64x64_161_170.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_161_170.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 161 - 170 : 
Epoch 161/170
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 65.15072226524353 secs
INFO:tensorflow:Setting weights on TPU model.
666/782 [========================>.....] - ETA: 3:39 - loss: 1.6115 - acc: 0.6362INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4

**Training with 64*64 epoch 171-180** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_161_170.hdf5")
last_executed_epoch = 170
nb_epoch = 180
csv_logger = CSVLogger('Assignment4_V36_64x64_171_180.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_171_180.hdf5")
!cp "Assignment4_V36_64x64_171_180.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_171_180.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 171 - 180 : 
Epoch 171/180
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 67.9875226020813 secs
INFO:tensorflow:Setting weights on TPU model.
361/782 [============>.................] - ETA: 14:40 - loss: 1.5433 - acc: 0.6506INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4

**Training with 64*64 epoch 181-190** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_171_180.hdf5")
last_executed_epoch = 180
nb_epoch = 190
csv_logger = CSVLogger('Assignment4_V36_64x64_181_190.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_181_190.hdf5")
!cp "Assignment4_V36_64x64_181_190.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_181_190.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 181 - 190 : 
Epoch 181/190
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 66.21472811698914 secs
INFO:tensorflow:Setting weights on TPU model.
213/782 [=======>......................] - ETA: 22:32 - loss: 1.4617 - acc: 0.6711INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(

**Training with 64*64 epoch 191-200** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_181_190.hdf5")
last_executed_epoch = 190
nb_epoch = 200
csv_logger = CSVLogger('Assignment4_V36_64x64_191_200.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_191_200.hdf5")
!cp "Assignment4_V36_64x64_191_200.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_191_200.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 191 - 200 : 
Epoch 191/200
INFO:tensorflow:Setting weights on TPU model.
782/782 [==============================] - 1361s 2s/step - loss: 1.4449 - acc: 0.6718 - val_loss: 2.3480 - val_acc: 0.5739
Epoch 192/200
782/782 [==============================] - 1340s 2s/step - loss: 1.4395 - acc: 0.6728 - val_loss: 2.2154 - val_acc: 0.5901
Epoch 193/200
782/782 [==============================] - 1340s 2s/step - loss: 1.4303 - acc: 0.6750 - val_loss: 2.2923 - val_acc: 0.5781
Epoch 194/200
782/782 [==============================] - 1340s 2s/step - loss: 1.4299 - acc: 0.6749 - val_loss: 2.4737 - val_acc: 0.5566
Epoch 195/200
782/782 [==============================] - 1337s 2s/step - loss: 1.4253 - acc: 0.6777 - val_loss: 2.3731 - val_acc: 0.5747
Epoch 196/200
782/782 [==============================] - 1343s 2s/step - loss: 1.4108 - acc: 0.6791 - val_loss: 2.2725 - val_acc: 0.5830
Epoch 197/200
782/782 [==============================] - 1340s 2s/step - loss: 1.4073 

**Training with 64*64 epoch 201-210** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_191_200.hdf5")
last_executed_epoch = 200
nb_epoch = 210
csv_logger = CSVLogger('Assignment4_V36_64x64_201_210.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_201_210.hdf5")
!cp "Assignment4_V36_64x64_201_210.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_201_210.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 201 - 210 : 
Epoch 201/210
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 64.69607996940613 secs
INFO:tensorflow:Setting weights on TPU model.
632/782 [=======================>......] - ETA: 4:46 - loss: 1.3880 - acc: 0.6832INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(4

**Training with 64*64 epoch 211-220** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_201_210.hdf5")
last_executed_epoch = 210
nb_epoch = 220
csv_logger = CSVLogger('Assignment4_V36_64x64_211_220.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_211_220.hdf5")
!cp "Assignment4_V36_64x64_211_220.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_211_220.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 211 - 220 : 
Epoch 211/220
INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(16,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(16, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape=(16, 200), dtype=tf.float32, name='activation_40_target_30')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Remapping placeholder for input_1
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.
INFO:tensorflow:Started compiling
INFO:tensorflow:Finished compiling. Time elapsed: 66.36090874671936 secs
INFO:tensorflow:Setting weights on TPU model.
 29/782 [>.............................] - ETA: 1:27:19 - loss: 1.2552 - acc: 0.7198INFO:tensorflow:New input shapes; (re-)compiling: mode=train (# of cores 8), [TensorSpec(shape=(4,), dtype=tf.int32, name='core_id0'), TensorSpec(shape=(4, 64, 64, 3), dtype=tf.float32, name='input_1_10'), TensorSpec(shape

**Training with 64*64 epoch 221-230** 

In [0]:
model.load_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_201_210.hdf5")
last_executed_epoch = 220
nb_epoch = 230
csv_logger = CSVLogger('Assignment4_V36_64x64_221_230.csv')

In [0]:
print('Training on 64*64 images: Epochs ' + str(last_executed_epoch+1) + ' - ' + str(nb_epoch) + " : ")
    
    
model.fit_generator(generator=train_generator_64,
                        steps_per_epoch=STEP_SIZE_TRAIN_64,
                        validation_data=valid_generator_64,
                        validation_steps=STEP_SIZE_VALID_64,
                        epochs=nb_epoch,
                        initial_epoch=last_executed_epoch,verbose=1,
                        callbacks=[csv_logger])

model.save_weights("/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_221_230.hdf5")
!cp "Assignment4_V36_64x64_221_230.csv" "/content/gdrive/My Drive/Assignment_4/Training_Backup/Assignment4_V36_64x64_221_230.csv"
print('-----------------------------------------Model weights and Logs saved to Google Drive-----------------------------------------')


Training on 64*64 images: Epochs 221 - 230 : 
Epoch 221/230
782/782 [==============================] - 1331s 2s/step - loss: 1.3072 - acc: 0.7033 - val_loss: 2.5543 - val_acc: 0.5601
Epoch 222/230
782/782 [==============================] - 1327s 2s/step - loss: 1.2955 - acc: 0.7041 - val_loss: 2.4611 - val_acc: 0.5697
Epoch 223/230
782/782 [==============================] - 1330s 2s/step - loss: 1.2951 - acc: 0.7056 - val_loss: 2.3083 - val_acc: 0.5946
Epoch 224/230
782/782 [==============================] - 1329s 2s/step - loss: 1.2910 - acc: 0.7075 - val_loss: 2.4643 - val_acc: 0.5825
Epoch 225/230
782/782 [==============================] - 1329s 2s/step - loss: 1.2846 - acc: 0.7082 - val_loss: 2.3944 - val_acc: 0.5825
Epoch 226/230
782/782 [==============================] - 1330s 2s/step - loss: 1.2761 - acc: 0.7106 - val_loss: 2.3979 - val_acc: 0.5903
Epoch 227/230
782/782 [==============================] - 1331s 2s/step - loss: 1.2778 - acc: 0.7104 - val_loss: 2.4346 - val_acc: 0.

# Final Evaluation

**The best accuracy is 59.49% in epoch 198**